### Another shot at dimensionality reduction techniques
Want to try PCA again with the following features: 
- Try on either HC or OFC only cells, small number (19 in HC, 18 in OFC)
- Condition on one selected feature at a time
- Group trials into 3 groups: 
  - A: high feature val, high confidence
  - B: low feature val, high confidence
  - C: low feature val, low confidence
Also, will want to try: 
- 50ms time bins, smoothed with 50ms std Gaussian

### Load Data, Imports

In [46]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import utils.pseudo_utils as pseudo_utils
import utils.pseudo_classifier_utils as pseudo_classifier_utils
import utils.behavioral_utils as behavioral_utils
from utils.session_data import SessionData
import utils.io_utils as io_utils
from utils.constants import *
import json

from spike_tools import (
    general as spike_general,
    analysis as spike_analysis,
)

import matplotlib.pyplot as plt
import matplotlib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
# the output directory to store the data
OUTPUT_DIR = "/data/patrick_res/pseudo"
# path to a dataframe of sessions to analyze
# SESSIONS_PATH = "/data/patrick_scratch/multi_sess/valid_sessions.pickle"
SESSIONS_PATH = "/data/patrick_res/sessions/valid_sessions_rpe.pickle"
# path for each session, specifying behavior
SESS_BEHAVIOR_PATH = "/data/rawdata/sub-SA/sess-{sess_name}/behavior/sub-SA_sess-{sess_name}_object_features.csv"
# path for each session, for spikes that have been pre-aligned to event time and binned. 
SESS_SPIKES_PATH = "/data/patrick_res/firing_rates/{sess_name}_firing_rates_{pre_interval}_{event}_{post_interval}_{interval_size}_bins_1_smooth.pickle"

FEATURE_DIMS = ["Color", "Shape", "Pattern"]

### Per session, label trials
Need confidence values, as well as feature values

In [64]:
def get_labels_for_session(session, feat):
    behavior_path = SESS_BEHAVIOR_PATH.format(sess_name=session)

    beh = pd.read_csv(behavior_path)
    valid_beh = behavioral_utils.get_valid_trials(beh)
    feature_selections = behavioral_utils.get_selection_features(valid_beh)
    valid_beh_merged = pd.merge(valid_beh, feature_selections, on="TrialNumber", how="inner")
    feat_dim = FEATURE_TO_DIM[feat]
    valid_beh_merged = valid_beh_merged[valid_beh_merged[feat_dim] == feat]
    valid_beh_vals = behavioral_utils.get_feature_values_per_session(session, valid_beh_merged)
    valid_beh_vals_conf = behavioral_utils.get_rpes_per_session(session, valid_beh_vals)
    med_conf = np.median(valid_beh_vals_conf["Prob_FE"].to_numpy())
    def assign_conf(row, med):
        row["Conf"] = "high" if row["Prob_FE"] > med else "low"
        return row
    valid_beh_vals_conf = valid_beh_vals_conf.apply(lambda row: assign_conf(row, med_conf), axis=1)

    # feat_dim = FEATURE_TO_DIM[feat]
    # selected_feat_beh = valid_beh_vals_conf[valid_beh_vals_conf[feat_dim] == feat]
    valid_beh_vals_conf["MaxFeatMatches"] = valid_beh_vals_conf.MaxFeat == feat
    valid_beh_vals_conf["Session"] = session
    return valid_beh_vals_conf

In [65]:
feature = "CYAN"
valid_sessions = pd.read_pickle(SESSIONS_PATH)
res = pd.concat(valid_sessions.apply(lambda row: get_labels_for_session(row.session_name, feature), axis=1).values)

In [67]:
res.groupby(["Session", "Conf", "MaxFeatMatches"]).count()[:50]

TrialNumber  BlockNumber  \
Session      Conf MaxFeatMatches                             
20180705     high False                    81           81   
                  True                     80           80   
             low  False                   106          106   
                  True                     56           56   
20180709     high False                    97           97   
                  True                     32           32   
             low  False                    91           91   
                  True                     38           38   
20180712     high False                    42           42   
                  True                     14           14   
             low  False                    43           43   
                  True                     14           14   
201807250001 high False                    20           20   
                  True                     32           32   
             low  False                    36           36   
                  True                     16           16   
20180801     high False                    68           68   
                  True                     73           73   
             low  False                    85           85   
                  True                     56           56   
20180802     high False                   122          122   
                  True                    114          114   
             low  False                   142          142   
                  True                     94           94   
20180803     high False                   131          131   
                  True                     72           72   
             low  False                   139          139   
                  True                     64           64   
20180806     high False                   106          106   
                  True                     32           32   
             low  False                   108          108   
                  True                     31           31   
20180808     high False                   111          111   
                  True                     21           21   
             low  False                   100          100   
                  True                     33           33   
20180910     high False                    62           62   
                  True                     67           67   
             low  False                    83           83   
                  True                     47           47   
20180912     high False                    71           71   
                  True                     72           72   
             low  False                   120          120   
                  True                     24           24   
20180917     high False                    73           73   
                  True                      5            5   
             low  False                    64           64   
                  True                     14           14   
20180918     high False                    67           67   
                  True                     51           51   

                                  TrialAfterRuleChange  TaskInterrupt  \
Session      Conf MaxFeatMatches                                        
20180705     high False                             81              1   
                  True                              80              0   
             low  False                            106              1   
                  True                              56              0   
20180709     high False                             97              0   
                  True                              32              0   
             low  False                             91              0   
                  True                              38              0   
20180712     high False                             42              0   
      